<a href="https://colab.research.google.com/github/RajeevRanjany/100DaysML/blob/main/WorkOutRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import pandas as pd
import os

In [39]:
path = "/root/.cache/kagglehub/datasets/niharika41298/gym-exercise-data/versions/1"
df = pd.read_csv(path + "/megaGymDataset.csv")
df.head()


,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [40]:
df.columns

Index(['Unnamed: 0', 'Title', 'Desc', 'Type', 'BodyPart', 'Equipment', 'Level',
       'Rating', 'RatingDesc'],
      dtype='object')

In [41]:
# cleanining and renaming column
df = df.rename(columns={
    "Title": "name",
    "Desc": "instructions",
    "BodyPart": "primary_muscle",
    "Equipment": "equipment",
    "Level": "difficulty",
    "Type": "workout_type"
})



In [42]:
# drop unwanted and fill na
df = df.drop("Unnamed: 0", axis=1)
df.fillna('', inplace=True)


/tmp/ipython-input-797954896.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [43]:
# creating tags
df["tags"] = (
    df["primary_muscle"] + " " +
    df["equipment"] + " " +
    df["difficulty"] + " " +
    df["workout_type"] + " " +
    df["instructions"]
).str.lower()


In [44]:
# creating final dataframe
new = df[["name", "tags"]]
new.head()

,name,tags
0,Partner plank band row,abdominals bands intermediate strength the par...
1,Banded crunch isometric hold,abdominals bands intermediate strength the ban...
2,FYR Banded Plank Jack,abdominals bands intermediate strength the ban...
3,Banded crunch,abdominals bands intermediate strength the ban...
4,Crunch,abdominals bands intermediate strength the cru...


In [45]:
df.isnull().sum()

,0
name,0
instructions,0
workout_type,0
primary_muscle,0
equipment,0
difficulty,0
Rating,0
RatingDesc,0
tags,0


In [46]:
# Vectorize
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

In [47]:
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)


In [71]:
def recommend(query_name):
    query_name = query_name.lower()  # make user input lowercase

    # first search in TAGS (best)
    matches = new[new['tags'].str.contains(query_name, case=False, regex=False)]

    # fallback to name search if needed
    if matches.empty:
        matches = new[new['name'].str.contains(query_name, case=False, regex=False)]

    if matches.empty:
        print(f"No exercise found for '{query_name}'. Try another keyword.")
        return

    index = matches.index[0]
    exercise_name = new.iloc[index]['name']

    distances = sorted(
        list(enumerate(similarity[index])),
        reverse=True,
        key=lambda x: x[1]
    )

    print(f"\nTop 5 exercises similar to '{exercise_name}':\n")
    for i in distances[1:6]:
        print(new.iloc[i[0]]["name"])


In [76]:
recommend("Bench Press")


Top 5 exercises similar to 'Band-suspended kettlebell bench press':

Band-suspended kettlebell bench press
Band-suspended kettlebell bench press
Board bench press
Close-grip bench press
Svend press


In [78]:
recommend('leg')


Top 5 exercises similar to 'FYR Banded Plank Jack':

Spider plank jack
Hand shift plank
Copenhagen plank
Lying cross-over lower back stretch
Push-up to side plank


In [75]:
recommend("Bench barbell roll-out")


Top 5 exercises similar to 'Bench barbell roll-out':

Barbell Side Bend
Barbell roll-out
Barbell Ab Rollout - On Knees
Barbell Ab Roll Out - Gethin Variation
30 Barbell Roll-Out


In [74]:
recommend('Barbell roll-out')


Top 5 exercises similar to 'Barbell roll-out':

Barbell Ab Rollout - On Knees
Barbell Ab Roll Out - Gethin Variation
30 Barbell Roll-Out
KV Barbell Hip Thrust
Bench barbell roll-out


In [73]:
recommend('KV Barbell Hip Thrust')


Top 5 exercises similar to 'KV Barbell Hip Thrust':

Shoulder-Elevated One-Leg Hip Thrusts
Holman Towel Reverse Plank Alternating Heel Pull-in
Holman Boat with Weighted Cross Punch
Holman Weighted Hook Punch
Holman Weighted Traveling Punch


In [55]:
df['instructions'].head()


,instructions
0,The partner plank band row is an abdominal exe...
1,The banded crunch isometric hold is an exercis...
2,The banded plank jack is a variation on the pl...
3,The banded crunch is an exercise targeting the...
4,The crunch is a popular core exercise targetin...


In [60]:
new['name'].head(20)


,name
0,Partner plank band row
1,Banded crunch isometric hold
2,FYR Banded Plank Jack
3,Banded crunch
4,Crunch
5,Decline band press sit-up
6,FYR2 Banded Frog Pump
7,Band low-to-high twist
8,Barbell roll-out
9,Barbell Ab Rollout - On Knees


In [70]:
df['primary_muscle'].unique()

array(['Abdominals', 'Adductors', 'Abductors', 'Biceps', 'Calves',
       'Chest', 'Forearms', 'Glutes', 'Hamstrings', 'Lats', 'Lower Back',
       'Middle Back', 'Traps', 'Neck', 'Quadriceps', 'Shoulders',
       'Triceps'], dtype=object)

In [72]:
recommend("chest")
recommend("Chest")
recommend("CHEST")



Top 5 exercises similar to 'Two-way swing':

Kettlebell swing
Dumbbell skier
Single-arm kettlebell snatch
Kettlebell deadlift
Single-arm kettlebell swing

Top 5 exercises similar to 'Two-way swing':

Kettlebell swing
Dumbbell skier
Single-arm kettlebell snatch
Kettlebell deadlift
Single-arm kettlebell swing

Top 5 exercises similar to 'Two-way swing':

Kettlebell swing
Dumbbell skier
Single-arm kettlebell snatch
Kettlebell deadlift
Single-arm kettlebell swing
